In [1]:
url = "https://github.com/crux82/squad-it/raw/master/"

In [2]:
data_files = {
    "train":url+"SQuAD_it-train.json.gz",
    "test":url+"SQuAD_it-test.json.gz"
}

In [4]:
from datasets import load_dataset

In [6]:
ds = load_dataset("json",data_files=data_files,field="data")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [7]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

In [10]:
import zipfile

In [1]:
import requests
zip_file_path = "https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip"

response = requests.get(zip_file_path)

In [6]:
with open("zip_file.zip", "wb") as f:
    f.write(response.content)

In [8]:
import zipfile
extract_to_path = "files"
with zipfile.ZipFile("zip_file.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)


In [9]:
data_files = {"train":"files/drugsComTrain_raw.tsv",
             "test":"files/drugsComTest_raw.tsv"}

In [49]:
from datasets import load_dataset
data = load_dataset("csv",data_files=data_files,delimiter="\t")

In [23]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [12]:
sample_ds = data["train"].shuffle(seed=42).select(range(500))

In [13]:
sample_ds

Dataset({
    features: ['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
    num_rows: 500
})

In [14]:
sample_ds= sample_ds.rename_column("Unnamed: 0","id")

In [15]:
sample_ds[0]

{'id': 87571,
 'drugName': 'Naproxen',
 'condition': 'Gout, Acute',
 'review': '"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
 'rating': 9.0,
 'date': 'September 2, 2015',
 'usefulCount': 36}

In [17]:
review_range = [len(i["review"]) for i in sample_ds]

In [19]:
max(review_range), min(review_range)

(1005, 7)

In [29]:
def lower_case(example):
    new_value = {"condition": example["condition"].lower()}
    return new_value
    

In [21]:
sample_ds.map(lower_case)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'lower'

In [22]:
sample_ds = sample_ds.filter(lambda x: x["condition"] is not None)

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [23]:
len(sample_ds)

496

In [30]:
sample_ds=sample_ds.map(lower_case)

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [31]:
sample_ds[0]

{'id': 87571,
 'drugName': 'Naproxen',
 'condition': 'gout, acute',
 'review': '"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
 'rating': 9.0,
 'date': 'September 2, 2015',
 'usefulCount': 36}

In [32]:
def add_review_length(example):
    new_value = {"review_length": len(example["review"])}
    return new_value

In [33]:
sample_ds = sample_ds.map(add_review_length)

Map:   0%|          | 0/496 [00:00<?, ? examples/s]

In [34]:
sample_ds[0]

{'id': 87571,
 'drugName': 'Naproxen',
 'condition': 'gout, acute',
 'review': '"like the previous person mention, I&#039;m a strong believer of aleve, it works faster for my gout than the prescription meds I take. No more going to the doctor for refills.....Aleve works!"',
 'rating': 9.0,
 'date': 'September 2, 2015',
 'usefulCount': 36,
 'review_length': 192}

In [50]:
data = data.filter(lambda x: x["condition"] is not None)

In [51]:
def filter_conditions(example):
    new_value = {
        "condition": example["condition"].lower(),
        "review_length": len(example["review"])
    }
    return new_value

In [52]:
data = data.map(filter_conditions)

In [41]:
#data = data.rename_column("Unnamed: 0","id")

In [53]:
data["train"][0]

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 79}

In [54]:
data = data.filter(lambda x: len(x["review"]) > 30)

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [55]:
data['train'][0]

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 79}

In [60]:
import html
mod_data = data.map(lambda x: {"review":[html.unescape(o) for o in x["review"]]} , batched=True)

Map:   0%|          | 0/158429 [00:00<?, ? examples/s]

Map:   0%|          | 0/52825 [00:00<?, ? examples/s]

In [61]:
mod_data['train'][0]

{'Unnamed: 0': 206461,
 'drugName': 'Valsartan',
 'condition': 'left ventricular dysfunction',
 'review': '"It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil"',
 'rating': 9.0,
 'date': 'May 20, 2012',
 'usefulCount': 27,
 'review_length': 79}

In [62]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [63]:
def tokenizer_split(example):
    return tokenizer(example["review"], truncation=True,max_length=128,
        return_overflowing_tokens=True,)

In [83]:
[len(i) for i in tokenizer_split(mod_data['train'][1])['input_ids']]

[128, 49]

In [95]:
tokenizer_split(mod_data['train'][1])

{'input_ids': [[101, 107, 1422, 1488, 1110, 9079, 1194, 1117, 2223, 1989, 1104, 1130, 19972, 11083, 119, 1284, 1245, 4264, 1165, 1119, 1310, 1142, 1314, 1989, 117, 1165, 1119, 1408, 1781, 1103, 2439, 13753, 1119, 1209, 1129, 1113, 119, 1370, 1160, 1552, 117, 1119, 1180, 6374, 1243, 1149, 1104, 1908, 117, 1108, 1304, 172, 14687, 1183, 117, 1105, 7362, 1111, 2212, 129, 2005, 1113, 170, 2797, 1313, 1121, 1278, 12020, 113, 1304, 5283, 1111, 1140, 119, 114, 146, 1270, 1117, 3995, 1113, 6356, 2106, 1105, 1131, 1163, 1106, 6166, 1122, 1149, 170, 1374, 1552, 119, 3969, 1293, 1119, 1225, 1120, 1278, 117, 1105, 1114, 2033, 1146, 1107, 1103, 2106, 119, 1109, 1314, 1160, 1552, 1138, 1151, 2463, 1714, 119, 1124, 1110, 150, 21986, 3048, 1167, 5340, 1895, 1190, 1518, 102], [101, 119, 1124, 1110, 1750, 6438, 113, 170, 1363, 1645, 114, 117, 1750, 172, 14687, 1183, 119, 1124, 1110, 11566, 1155, 1103, 1614, 1119, 1431, 119, 8007, 1117, 4658, 1110, 1618, 119, 1284, 1138, 1793, 1242, 1472, 23897, 1105, 117

In [89]:
mod_data["train"].column_names

['Unnamed: 0',
 'drugName',
 'condition',
 'review',
 'rating',
 'date',
 'usefulCount',
 'review_length']

In [90]:
tokenized_ds = mod_data.map(tokenizer_split,batched=True,remove_columns=mod_data["train"].column_names)

Map:   0%|          | 0/158429 [00:00<?, ? examples/s]

Map:   0%|          | 0/52825 [00:00<?, ? examples/s]

In [92]:
len(mod_data["train"]), len(mod_data["test"])

(158429, 52825)

In [91]:
len(tokenized_ds["train"]), len(tokenized_ds["test"])

(226687, 75593)

In [98]:
mod_data['train'][1]

{'Unnamed: 0': 95260,
 'drugName': 'Guanfacine',
 'condition': 'adhd',
 'review': '"My son is halfway through his fourth week of Intuniv. We became concerned when he began this last week, when he started taking the highest dose he will be on. For two days, he could hardly get out of bed, was very cranky, and slept for nearly 8 hours on a drive home from school vacation (very unusual for him.) I called his doctor on Monday morning and she said to stick it out a few days. See how he did at school, and with getting up in the morning. The last two days have been problem free. He is MUCH more agreeable than ever. He is less emotional (a good thing), less cranky. He is remembering all the things he should. Overall his behavior is better. \r\nWe have tried many different medications and so far this is the most effective."',
 'rating': 8.0,
 'date': 'April 27, 2010',
 'usefulCount': 192,
 'review_length': 741}

In [116]:
def tokenize_split(examples):
    result = tokenizer(examples["review"], max_length=128, truncation=True, return_overflowing_tokens = True)
    overlfow = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in overlfow]
    return result
    

In [117]:
tokenized_ds = mod_data.map(tokenize_split,batched=True)

Map:   0%|          | 0/158429 [00:00<?, ? examples/s]

Map:   0%|          | 0/52825 [00:00<?, ? examples/s]

In [118]:
len(tokenized_ds["train"])

226687